In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [172]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 15
EVENT_NAME = 'EMA_CROSS_8_34'
LABEL_NAME = "TB"

START_DATE = "20240101"
END_DATE = "20241231"

RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_FEATURE.pkl"
EVENT_FILE_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}.pkl"

In [173]:
DIRECTION_LABEL_DIR = BASE_DIR / "direction_labels"
DIRECTION_LABEL_DIR.mkdir(parents=True, exist_ok=True)
DIRECTION_LABEL_FILE_PATH = DIRECTION_LABEL_DIR / f"{RESAMPLED_NAME}-{EVENT_NAME}-{LABEL_NAME}.pkl"

In [174]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)
t_events = pd.read_pickle(EVENT_FILE_PATH)

CPU times: user 2.56 ms, sys: 11.5 ms, total: 14 ms
Wall time: 13.2 ms


In [175]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'return',
       'log_return', 'fd_return', 'log_fd_return', 'SMA_5', 'SMA_15', 'SMA_30',
       'SMA_60', 'EMA_5', 'EMA_15', 'EMA_30', 'EMA_60', 'MACD_12_26_9',
       'MACDh_12_26_9', 'MACDs_12_26_9', 'RSI_5', 'RSI_14', 'RSI_20', 'ADX_5',
       'ADXR_5_2', 'DMP_5', 'DMN_5', 'ADX_15', 'ADXR_15_2', 'DMP_15', 'DMN_15',
       'ADX_30', 'ADXR_30_2', 'DMP_30', 'DMN_30', 'ADX_60', 'ADXR_60_2',
       'DMP_60', 'DMN_60', 'ATRr_5', 'ATRr_10', 'ATRr_15', 'ATRr_20',
       'BBL_5_2.0_2.0', 'BBM_5_2.0_2.0', 'BBU_5_2.0_2.0', 'BBB_5_2.0_2.0',
       'BBP_5_2.0_2.0', 'BBL_14_2.0_2.0', 'BBM_14_2.0_2.0', 'BBU_14_2.0_2.0',
       'BBB_14_2.0_2.0', 'BBP_14_2.0_2.0'],
      dtype='object')

In [176]:
df.loc[t_events].head()

,open,high,low,close,volume,spread,return,log_return,fd_return,log_fd_return,...,BBL_5_2.0_2.0,BBM_5_2.0_2.0,BBU_5_2.0_2.0,BBB_5_2.0_2.0,BBP_5_2.0_2.0,BBL_14_2.0_2.0,BBM_14_2.0_2.0,BBU_14_2.0_2.0,BBB_14_2.0_2.0,BBP_14_2.0_2.0
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-02-19 07:15:00,1.078825,1.07895,1.078660,1.078685,2.587290e+12,0.000025,-0.000145,-0.000134,0.154695,0.011132,...,1.078058,1.078536,1.079014,0.088726,0.655704,1.077423,1.078156,1.078890,0.136139,0.860112
2024-02-19 09:00:00,1.077830,1.07835,1.077715,1.077720,5.750420e+12,0.000023,-0.000105,-0.000097,0.153883,0.010379,...,1.077439,1.078215,1.078991,0.143858,0.180872,1.077729,1.078364,1.078999,0.117741,-0.007161
2024-02-19 18:00:00,1.077610,1.07798,1.077610,1.077980,8.185700e+11,0.000026,0.000360,0.000334,0.154587,0.011032,...,1.077021,1.077552,1.078083,0.098470,0.903369,1.076028,1.076989,1.077950,0.178387,1.015859
2024-02-20 00:00:00,1.077545,1.07770,1.077305,1.077310,1.781690e+12,0.000027,-0.000245,-0.000227,0.153732,0.010238,...,1.077286,1.077529,1.077772,0.045152,0.049867,1.077334,1.077817,1.078301,0.089756,-0.024600
2024-02-20 07:00:00,1.077100,1.07749,1.077090,1.077360,5.246670e+12,0.000025,0.000265,0.000246,0.154149,0.010625,...,1.076942,1.077174,1.077406,0.043152,0.900149,1.076616,1.076975,1.077335,0.066808,1.034596


## Calculate Target Volatility (trgt)
This will be used for the calculation of dynamic TP/SL

In [177]:
def get_daily_vol(close,span0=100):
    # daily vol reindexed to close
    df0=close.index.searchsorted(close.index-pd.Timedelta(days=1))
    #bp()
    df0=df0[df0>0]
    #bp()
    df0=(pd.Series(close.index[df0-1],
                   index=close.index[close.shape[0]-df0.shape[0]:]))
    #bp()
    try:
        df0=close.loc[df0.index]/close.loc[df0.values].values-1 # daily rets
    except Exception as e:
        print(e)
        print('adjusting shape of close.loc[df0.index]')
        cut = close.loc[df0.index].shape[0] - close.loc[df0.values].shape[0]
        df0=close.loc[df0.index].iloc[:-cut]/close.loc[df0.values].values-1
    df0=df0.ewm(span=span0).std().rename('dailyVol')
    return df0

In [178]:
from ta.volatility import AverageTrueRange

def get_atr(df, window=60):
    atr = AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=window)
    return atr.average_true_range().rename(f"atr{window}")

In [179]:
trgt = get_daily_vol(df['close']).reindex(t_events, method='ffill')

In [180]:
trgt = (df['ATRr_20'] / df['close']).shift(1).reindex(t_events, method='ffill') # rate

In [181]:
trgt.describe()

count    813.000000
mean       0.000440
std        0.000180
min        0.000166
25%        0.000309
50%        0.000391
75%        0.000521
max        0.001359
dtype: float64

## Apply Triple Barrier
We use daily volatility to calculate TP/SL, and we set our T1 as 1day

In [182]:
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import pandas as pd

def applyPtSlOnT1(close,events,ptSl,molecule):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    
    pt = ptSl[0] * events_['trgt'] if ptSl[0] > 0 else pd.Series(index=events.index)
    sl = -ptSl[1] * events_['trgt'] if ptSl[1] > 0 else pd.Series(index=events.index)

        
    for loc, t1 in events_['t1'].fillna(close.index[-1]).items():
        df0=close[loc:t1] # path prices
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns (rate)
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking
        
    return out


def parallel_apply(func, items, num_threads=4, **kwargs):
    
    def worker(molecule):
        return func(molecule=molecule, **kwargs)

    chunks = np.array_split(items, num_threads)
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(worker, chunks))

    return pd.concat(results).sort_index()


def getEvents(close, tEvents, ptSl, trgt, minTrgt, numThreads=4, t1=False, side=None):
    # Step 1: Filter targets
    trgt = trgt.loc[tEvents]
    trgt = trgt[trgt > minTrgt]

    # Step 2: Set vertical barrier (t1)
    if t1 is False:
        t1 = pd.Series(pd.NaT, index=tEvents)

    # Step 3: Build events DataFrame
    if side is None:
        side_, ptSl_ = pd.Series(1., index=trgt.index), [ptSl[0], ptSl[0]]
    else:
        side_, ptSl_ = side.loc[trgt.index], ptSl[:2]

    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1).dropna(subset=['trgt'])

    # Step 4: Apply barriers in parallel
    df0 = parallel_apply(
        func=applyPtSlOnT1,
        items=events.index,
        num_threads=numThreads,
        close=close,
        events=events,
        ptSl=ptSl_
    )

    # Step 5: Choose the first touched barrier
    events['t1'] = df0.dropna(how='all').min(axis=1)
    
    if side is None:
        events = events.drop('side', axis=1)
        
    return events


In [183]:
def get_vertical_barrier(t_events, close, delta=pd.Timedelta(minutes=60)):
    """
    For each event in t_events, find the timestamp in close.index 
    that is at least delta later. Returns a pd.Series of t1.
    """
    barrier_times = t_events + delta
    t1_idx = close.index.searchsorted(barrier_times)
    valid_idx = t1_idx[t1_idx < len(close)]
    t1 = pd.Series(close.index[valid_idx], index=t_events[:len(valid_idx)])
    return t1

In [208]:
t1 = get_vertical_barrier(t_events, df['close'], delta=pd.Timedelta(minutes=120))

In [209]:
t1

timestamp
2024-02-19 07:15:00   2024-02-19 09:15:00
2024-02-19 09:00:00   2024-02-19 11:00:00
2024-02-19 18:00:00   2024-02-19 20:00:00
2024-02-20 00:00:00   2024-02-20 02:00:00
2024-02-20 07:00:00   2024-02-20 09:00:00
                              ...        
2024-12-30 02:15:00   2024-12-30 04:15:00
2024-12-30 06:15:00   2024-12-30 08:15:00
2024-12-30 08:00:00   2024-12-30 10:00:00
2024-12-30 10:00:00   2024-12-30 12:00:00
2024-12-30 13:45:00   2024-12-30 15:45:00
Name: timestamp, Length: 812, dtype: datetime64[ns]

In [210]:
trgt.describe()

count    813.000000
mean       0.000440
std        0.000180
min        0.000166
25%        0.000309
50%        0.000391
75%        0.000521
max        0.001359
dtype: float64

In [211]:
minTrgt = trgt.quantile(.3)
# minTrgt = 0

In [212]:
%%time

events = getEvents(
    close=df['close'],
    tEvents=t_events, 
    ptSl=[1,1], 
    trgt=trgt, 
    minTrgt=minTrgt, 
    numThreads=20, 
    t1=t1, 
    side=None
)

CPU times: user 174 ms, sys: 7.45 ms, total: 182 ms
Wall time: 178 ms


In [213]:
events.head()

,t1,trgt
timestamp,,
2024-02-19 09:00:00,2024-02-19 11:00:00,0.000333
2024-02-19 18:00:00,2024-02-19 20:00:00,0.000338
2024-02-20 22:00:00,2024-02-21 00:00:00,0.000404
2024-02-20 22:45:00,2024-02-21 00:45:00,0.000370
2024-02-21 13:45:00,2024-02-21 15:00:00,0.000433


In [214]:
events = events.dropna().copy()

# Keep only intraday events

In [215]:
events = events[events.index.date==events.t1.dt.date]

## Create classification labels

In [216]:
def getBins(events, close, minRet=0.0, t1=None):
    '''
    Compute event's outcome (including side information, if provided).
    events is a DataFrame where:
    -events.index is event's starttime
    -events['t1'] is event's endtime
    -events['trgt'] is event's target
    -events['side'] (optional) implies the algo's position side
    -t1 is original vertical barrier series
    Case 1: ('side' not in events): bin in (-1,1) <-label by price action
    Case 2: ('side' in events): bin in (0,1) <-label by pnl (meta-labeling)
    '''
    # 1) prices aligned with events
    events_ = events.dropna(subset=['t1'])
    px = events_.index.union(events_['t1'].values).drop_duplicates()
    px = close.reindex(px, method='bfill')
    # 2) create out object
    out = pd.DataFrame(index=events_.index)
    out['ret'] = px.loc[events_['t1'].values].values / px.loc[
        events_.index] - 1
    if 'side' in events_: out['ret'] *= events_['side']  # meta-labeling
    out['bin'] = np.sign(out['ret'])

    if 'side' not in events_:
        # only applies when not meta-labeling.
        # to update bin to 0 when vertical barrier is touched, we need the
        # original vertical barrier series since the events['t1'] is the time
        # of first touch of any barrier and not the vertical barrier
        # specifically. The index of the intersection of the vertical barrier
        # values and the events['t1'] values indicate which bin labels needs
        # to be turned to 0.
        vtouch_first_idx = events[events['t1'].isin(t1.values)].index
        out.loc[vtouch_first_idx, 'bin'] = 0.

    if 'side' in events_: out.loc[out['ret'] <= 0, 'bin'] = 0  # meta-labeling
    out.loc[out['ret'].abs() < minRet, 'bin'] = 0. # 不做獲利過少的交易 (設為 0)
    # out = out.loc[out['ret'].abs() >= minRet] # 過濾掉獲利過少的交易 (刪掉)
    return out

In [217]:
labels = getBins(events, df['close'], minRet=0, t1=t1)

In [218]:
print(labels.ret.abs().describe())
print(labels.ret.abs().quantile(.3))

count    534.000000
mean       0.000691
std        0.000645
min        0.000000
25%        0.000416
50%        0.000587
75%        0.000841
max        0.009929
Name: ret, dtype: float64
0.0004485553765450234


In [219]:
labels.bin.value_counts()

bin
-1.0    194
 1.0    185
 0.0    155
Name: count, dtype: int64

In [220]:
labels["bin_class"] = labels["bin"] + 1

In [221]:
labeled_events = events.join(labels, how='inner')

In [222]:
labeled_events.head()

,t1,trgt,ret,bin,bin_class
timestamp,,,,,
2024-02-19 09:00:00,2024-02-19 11:00:00,0.000333,-0.000213,0.0,1.0
2024-02-19 18:00:00,2024-02-19 20:00:00,0.000338,-0.000121,0.0,1.0
2024-02-21 13:45:00,2024-02-21 15:00:00,0.000433,0.000657,1.0,2.0
2024-02-22 13:15:00,2024-02-22 13:30:00,0.000607,-0.000853,-1.0,0.0
2024-02-23 09:00:00,2024-02-23 09:30:00,0.000353,-0.000504,-1.0,0.0


## Save events to disk

In [223]:
labeled_events.to_pickle(DIRECTION_LABEL_FILE_PATH)

In [224]:
DIRECTION_LABEL_FILE_PATH

PosixPath('../data/direction_labels/USDJPY-15m-20240101-20241231-EMA_CROSS_8_34-TB.pkl')